# 핵심 키워드 추출 (Keyword Extraction)

In [ ]:
# Mecab 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
# mecab-python의 버전 오류로 인해 아래 패키지를 설치하면 코랩에서 Mecab을 사용가능
!pip install mecab-python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.6/581.6 kB 6.0 MB/s eta 0:00:00


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 패키지 업그레이드로 인한 에러 처리를 위해 라이브러리 버전 Downgrade
!pip install 'networkx<2.7'

# 런타임 재시작 !

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

# 1.1 TF-IDF 활용 핵심키워드 추출

## 실습 1. sklearn 활용

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_news_by_url(url):
    res = requests.get(url, headers = {
        'User-Agent': 'Mozilla/5.0'
    })
    bs = BeautifulSoup(res.content, 'html.parser')

    title = bs.select_one('#title_area').text #제목
    content = bs.select_one('#contents').get_text().replace('\n', " ") #본문
    content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    return  content.strip()

docs = []
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728') )
len(docs)

### 1) 전처리

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
    # 명사와 동사만으로 문서 전처리
    preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

### 2) TF-IDF 계산

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_df=0.85, max_features=10000)
word_count_vector = count_vectorizer.fit_transform(preprocessed_docs)
list(count_vectorizer.vocabulary_.keys())[:10]

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

### 3) 핵심키워드 추출

In [ ]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)

def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [ ]:
doc = preprocessed_docs[0] # 핵심키워드 추출할 문서 조회

feature_names = count_vectorizer.get_feature_names() # TF-IDF 단어 목록
tf_idf_vector = tfidf_transformer.transform(count_vectorizer.transform([doc])) # 문서의 tf-idf 추출
sorted_keywords = sort_keywords(tf_idf_vector.tocoo()) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)

print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)

In [ ]:
tf_idf_vector.tocoo().data


---


## 실습 2. gensim 활용


### 1) 전처리

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
    # 명사와 동사만으로 문서 전처리
    preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

### 2) TF-IDF 계산

In [ ]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

document_ls = [doc.split() for doc in preprocessed_docs]
dct = Dictionary(document_ls) # 인덱스(key) - 단어(value) 인 딕셔너리 생성
corpus = [dct.doc2bow(doc) for doc in document_ls] # 각 문서에 포함된 단어를 인덱스로 변환하여 corpus 생성
tfidf = TfidfModel(corpus) # TF-IDF 산출

### 3) 핵심키워드 추출

In [ ]:
def sort_keywords(tfidf):
    return sorted(tfidf, key=lambda x: (x[1], x[0]), reverse=True)

def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [ ]:
doc = corpus[0]

sorted_keywords = sort_keywords(tfidf[doc]) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(dct, sorted_keywords, 5)

print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)

In [ ]:
tfidf[doc]



---



# 1.2 Textrank
https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf

<img src="https://3.bp.blogspot.com/-yp0Lr3ec5EY/XIs6znCcO_I/AAAAAAAAAPY/xtZxe_OYtH0xeuWsp4Qd4DQrunGMpVQmQCLcBGAs/s640/keyword-extraction-textrank.png" />

## 실습 1. 행렬 활용


In [1]:
doc = '딸기 바나나 사과 딸기 파인애플'

### 1) 토큰화 (Tokenization)

분석 텍스트 정제

In [3]:
tokens = doc.split()
tokens

['딸기', '바나나', '사과', '딸기', '파인애플']

### 2) Unique한 토큰 목록 생성

그래프 생성을 위해서 Unique한 토큰 목록 생성

In [7]:
1 in [1, 3, 5]

True

In [9]:
# set(tokens)

unique_tokens = [] # 유니크한 토큰 목록을 담을 변수
for token in tokens: # 토큰들 하나씩 꺼내보기
    # print(token)
    # 있는지 없는지 체크
    have_token = token in unique_tokens # boolean
    if have_token:
        # 추가하지 않는다.
        pass
    else: # 없으면
        # 추가한다
        unique_tokens.append(token)

unique_tokens

['딸기', '바나나', '사과', '파인애플']

In [15]:
nodes = unique_tokens

In [16]:
nodes

['딸기', '바나나', '사과', '파인애플']

In [18]:
node2idx = {node: i for i, node in enumerate(nodes)}
node2idx

{'딸기': 0, '바나나': 1, '사과': 2, '파인애플': 3}

In [19]:
node_scores = [1.0] * len(nodes)
node_scores

[1.0, 1.0, 1.0, 1.0]

In [21]:
node_scores[ node2idx['딸기'] ]

1.0

In [22]:
tokens

['딸기', '바나나', '사과', '딸기', '파인애플']

In [28]:
temp = 'abc'

In [ ]:
temp[0] = 'd'

In [32]:
temp = ('123', '234') # immutable 바꿀수 없는 str, int, float
# mutable set, dict, list
temp[0] = '777'

TypeError: ignored

In [33]:
# 엣지(연결 선) 정보
window_size = 2
# edges = []
edges = { node: set() for node in nodes }

for window_start in range(len(tokens) - 1):
    # context
    start_token = tokens[window_start]
    end_token = tokens[window_start + 1]
    # list에 추가하는 방법
    # edges.append( (start_token, end_token) )
    # edges.append( (end_token, start_token) )

    # dict에 추가하는 방법
    edges[start_token].add(end_token) # set().add()
    edges[end_token].add(start_token)

# unique_edges = list(set(edges))
# unique_edges

edges

{'딸기': {'바나나', '사과', '파인애플'},
 '바나나': {'딸기', '사과'},
 '사과': {'딸기', '바나나'},
 '파인애플': {'딸기'}}

In [34]:
nodes, node_scores, edges

(['딸기', '바나나', '사과', '파인애플'],
 [1.0, 1.0, 1.0, 1.0],
 {'딸기': {'바나나', '사과', '파인애플'},
  '바나나': {'딸기', '사과'},
  '사과': {'딸기', '바나나'},
  '파인애플': {'딸기'}})

In [48]:
d = 0.85
def cal_score():
    new_scores = [0] * len(nodes)
    for target_node in nodes:
        edge_weight_sum = 0
        # 나에게 들어오는 노드들
        for in_node in edges[target_node]:
            # 몇개 노드와 연결?
            out_vj = len(edges[in_node])
            # 현재 스코어는?
            S_vj = node_scores[ node2idx[in_node] ]
            edge_weight = S_vj/out_vj

            edge_weight_sum += edge_weight

        S_vi = (1-d) + d * edge_weight_sum
        # print(S_vi)
        # new_scores.append(S_vi)
        new_scores[node2idx[target_node]] = S_vi
    return new_scores

node_scores = cal_score()
node_scores

[1.4669435480283939, 0.983711252789227, 0.983711252789227, 0.5656339463931519]

In [49]:
for epoch in range(30):
    node_scores = cal_score()
node_scores

[1.4669434685404483,
 0.9837112743532295,
 0.9837112743532295,
 0.5656339827530932]

In [ ]:
# 특정 노드의 Textrank 점수 = (1-d) + d * (나한테 들어오는 엣지 가중치들의 합)


### 3) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex)
*   weighted_edge 행렬은 노드간 가중치 정보를 담고 있음
*   weighted_edge[i][j] 는 i번째 단어와 j번째 단어의 가중치를 의미
*   weighted_edge[i][j] 가 0인 경우는 노드간 연결이 없음을 의미
*   모든 노드는 1로 초기화

In [ ]:
# 그래프 구조 잡기

# 무방향
    # 시작노드에 끝 노드 추가
    # 끝 노드에도 시작 노드 추가

# 1. window_size 만큼 토큰들을 가져온다 (context)
# 2. 2개씩 짝지어준다 (조합) -> 이중 반복문
# 3. edges에 추가해준다 (중복 x) set 자료형 활용


### 4) 각 노드의 score계산
각 노드와 연결된 weighted edge의 값을 합산

In [ ]:
# score 계산 로직

# 1. Textrank score 계산할 노드 선택 (반복문)
# 2. 해당 노드와 연결된 노드들 (반복문)
    # score, 연결된 노드 수를 통해 엣지 가중치 계산
    # 새로운 score를 new_scores에 담아준다
# 3. 계산이 끝나면 기존 노드 스코어 갱신


### 5) 핵심 단어 추출

In [53]:
import numpy as np
np.argsort(node_scores)[::-1]
idx2node

array([0, 2, 1, 3])

### 행렬 자료구조 활용

edges만 변경됨

In [56]:
edges = [[0] * len(nodes) for _ in nodes]
print(*edges, sep='\n')

[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]


In [62]:
# edges[start_node][end_node] => 연결된 상태 확인 가능
for window_start in range(len(tokens) - 1):
    start_node = tokens[window_start]
    end_node = tokens[window_start+1]
    # print(start_node, end_node)
    edges[node2idx[start_node]][node2idx[end_node]] = 1 # 연결 됨
    edges[node2idx[end_node]][node2idx[start_node]] = 1 # 연결 됨

print(*edges, sep='\n')

[0, 1, 1, 1]
[1, 0, 1, 0]
[1, 1, 0, 0]
[1, 0, 0, 0]


In [65]:
weight_edge_matrix = edges.copy()
weight_edge_matrix

[[0, 1, 1, 1], [1, 0, 1, 0], [1, 1, 0, 0], [1, 0, 0, 0]]

In [68]:
from copy import deepcopy
weight_edge_matrix = deepcopy(edges)

In [70]:
weight_edge_matrix[0][1] = 0
edges

[[0, 1, 1, 1], [1, 0, 1, 0], [1, 1, 0, 0], [1, 0, 0, 0]]

In [71]:
weight_edge_matrix

[[0, 0, 1, 1], [1, 0, 1, 0], [1, 1, 0, 0], [1, 0, 0, 0]]

In [74]:
node_scores = [1.0] * len(nodes)
weight_edge_matrix = deepcopy(edges)

for start_node in nodes:
    start_idx = node2idx[start_node]
    out_vi = sum(edges[start_idx])
    for end_node in nodes:
        end_idx = node2idx[end_node]
        weight_edge_matrix[start_idx][end_idx] = edges[start_idx][end_idx] * node_scores[start_idx] / out_vi # 엣지 가중치 = 스코어 / 내가 연결된 개수
#
weight_edge_matrix

[[0.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
 [0.5, 0.0, 0.5, 0.0],
 [0.5, 0.5, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0]]

In [75]:
new_scores = []
for node in nodes:
    node_idx = node2idx[node]
    weight_edge_sum = 0
    for row in weight_edge_matrix:
        weight_edge_sum += row[node_idx]
    new_scores.append((1-d)+d*weight_edge_sum)

new_scores

[1.85, 0.8583333333333333, 0.8583333333333333, 0.43333333333333335]

### numpy array

In [77]:
nodes, node_scores, edges

node_scores = np.array(node_scores)
edges = np.array(edges)

In [78]:
edges

array([[0, 1, 1, 1],
       [1, 0, 1, 0],
       [1, 1, 0, 0],
       [1, 0, 0, 0]])

In [82]:
edges.sum(axis=0)

array([3, 2, 2, 1])

In [83]:
weight_edge_matrix = edges * node_scores.reshape(-1, 1) / edges.sum(axis=0).reshape(-1, 1)
weight_edge_matrix

array([[0.        , 0.33333333, 0.33333333, 0.33333333],
       [0.5       , 0.        , 0.5       , 0.        ],
       [0.5       , 0.5       , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        ]])

In [88]:
new_scores = 1 - d + d*weight_edge_matrix.sum(axis=0)

In [89]:
new_scores

array([1.85      , 0.85833333, 0.85833333, 0.43333333])

In [91]:
for epoch in range(30):
    weight_edge_matrix = edges * node_scores.reshape(-1, 1) / edges.sum(axis=0).reshape(-1, 1)
    new_scores = 1 - d + d * weight_edge_matrix.sum(axis=0)
    node_scores = new_scores

node_scores

array([1.46694347, 0.98371127, 0.98371127, 0.56563398])

---

## 실습 2. 그래프 활용

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_news_by_url(url):
    res = requests.get(url, headers = {
        'User-Agent': 'Mozilla/5.0'
    })
    bs = BeautifulSoup(res.content, 'html.parser')

    title = bs.select_one('#title_area').text #제목
    content = bs.select_one('#contents').get_text().replace('\n', " ") #본문
    content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    return content.strip()

doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc[:50]

### 1) 토큰화 (Tokenization)

분석 텍스트 정제

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

nodes = [token for token in mecab.pos(doc) if token[1] in ['NNG','NNP']] #NNG, NNP를 스코어 계산 대상(노드)로 제한
tokens = [token for token in mecab.pos(doc)] #탐색할 토큰 전체

### 2) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 없음을 의미
*   모든 노드는 1로 초기화

In [ ]:
import numpy as np
import math
import networkx as nx

# 윈도내 동시 등장한 토큰으로 그래프를 생성
def connect(nodes, tokens) :
    window_size = 2 # coocurrence를 판단하기 위한 window 사이즈 설정

    edges = []
    for window_start in range(0,(len(tokens)-window_size+1)):
        window = tokens[window_start:window_start+window_size]
        for i in range(window_size) :
            for j in range(i+1, window_size) :
                if (window[i] in nodes) & (window[j] in nodes) : #윈도 내 단어가 노드에 속하는 경우만 엣지로 연결
                    edges.append((window[i], window[j]))
                    print((window[i], window[j]))
    return edges

# tokens = ['딸기', '바나나', '사과', '딸기', '파인애플']
# nodes = ['바나나', '사과', '파인애플', '딸기']

graph=nx.diamond_graph()
graph.clear()
graph.add_nodes_from(list(set(nodes))) #node 등록
graph.add_edges_from(connect(nodes, tokens)) #edge 연결

### 3) 스코어 계산 및 핵심키워드 추출

In [ ]:
scores = nx.pagerank(graph) #pagerank 계산
rank = sorted(scores.items(), key=lambda x: x[1], reverse=True) #score 역순 정렬
print("\n=== 핵심키워드 ===")
rank[:5]

In [ ]:
import networkx as nx
from IPython.core.display import Image
from networkx.drawing.nx_pydot import to_pydot

d = to_pydot(graph)
d.set_dpi(600)
d.set_rankdir("LR")
Image(d.create_png(), width=600)

---

## 실습 3. TextRank 핵심 구 추출

### 1) 불용어를 기준으로 구 추출

In [ ]:
phrases = []
phrase = ' '

for word in tokens:
    if word[1][0] != 'N':
        if phrase != ' ':
            phrases.append(phrase.strip())
        phrase= ' '
    else:
        phrase+=word[0]
        phrase+= ' '

print(phrases)

In [ ]:
unique_phrases = []

for phrase in phrases:
    if phrase not in unique_phrases:
        unique_phrases.append(phrase)

print(unique_phrases)

### 2) 각 구의 Score 계산

앞서 산출한 각 단어별 점수를 합산

In [ ]:
vocabs = dict((r[0][0], r[1]) for r in rank)
phrase_scores = []
keywords = []

for phrase in unique_phrases:
    phrase_score = 0
    for word in phrase.split():
        if word in vocabs.keys():
            phrase_score+=vocabs[word]

    phrase_scores.append((phrase, phrase_score))

for phrase_score in phrase_scores[:10]:
    print(f"keyword: {phrase_score[0]}, Score : {phrase_score[1]}")

### 3) 각 구를 Score로 정렬하여 핵심 구 추출

In [ ]:
sorted_phrase_score = sorted(phrase_scores, key=lambda tup: tup[1], reverse=True)
sorted_phrase_score[:10]



---



## 실습 4. gensim 활용

- summarization.keywords(document, words) : document를 요약하여 words만큼 키워드를 추출

In [ ]:
from gensim.summarization import keywords
keywords(doc, words=5).split('\n')